In [ ]:
### Import packages for authentication

from google.cloud import bigquery
from google.oauth2 import service_account

### Import packages for converting query results into dataframe

import pandas as pd 

## Import packages to create absolute file path &  make code independent of operating system

from pathlib import Path
import os.path

### Import packages to visualize data

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
### Authentication 

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/GoogleBigQuery_key.json").resolve()

credentials = service_account.Credentials.from_service_account_file(os.path.join(full_path))

In [ ]:
### Construct a BigQuery client object.

client = bigquery.Client(credentials=credentials, 
project = credentials.project_id)

In [ ]:
### Getting overview of Stackoverflow tables

stackoverflow = client.dataset('stackoverflow', project= 'bigquery-public-data')
print([x.table_id for x in client.list_tables(stackoverflow)])

In [ ]:
### Make an API request

query = """
SELECT
      pq.score as question_score, pa.score as answer_score, pq.id as question_id, pa.parent_id as question_id_check, pq.title as question_title, pq.body       as question_text, pq.answer_count, pq.comment_count, pq.creation_date, pq.tags, pq.view_count, pa.body as answer_text
FROM `bigquery-public-data.stackoverflow.posts_questions` pq
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` pa ON pq.id = pa.parent_id
WHERE pa.creation_date > "2019-05-30 00:00:00.000 UTC"
"""

dataframe = (
    client.query(query)
    .result()
    .to_dataframe()
)

In [ ]:
len(dataframe)

In [ ]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw.csv").resolve()

dataframe.to_csv(os.path.join(full_path))

In [ ]:
### Temporary code to quickly open dataset

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw.csv").resolve()

dataframe = pd.read_csv(os.path.join(full_path))

In [ ]:
### Display query results

display(dataframe)

In [ ]:
### Check for null values

dataframe.isnull().values.any()
dataframe.isnull().sum()

In [ ]:
#### Histogram of distribution of the answer scores

f, ax = plt.subplots(figsize=(40,40))
sns.countplot(x='answer_score', data=dataframe)
plt.xlim(None, 100) 
plt.ylim(0, 9000) 
plt.show()

In [ ]:
#### Histogram of distribution of the number of views of the question & answer

f, ax = plt.subplots(figsize=(40,40))
sns.countplot(x='view_count', data=dataframe)
plt.xlim(None, 200) 
plt.ylim(0, None) 
plt.show()

In [ ]:
### Summarization of variables for only numeric

dataframe.describe()

# mean score is 0.83 while 50% percentile / median score is 0.
# 25th percentile for view count is 39 views, median is 64 views;

In [ ]:
dataframe['answer_score'].value_counts() # .sort_index()

In [ ]:
### Drop all datarows with answers that have gotten fewer than 39 views (25th percentile in view_count distribution)

dataframe_views = dataframe.drop(dataframe[dataframe.view_count < 39].index)
len(dataframe_views)

In [ ]:
### Save dataframe with only > 39 views to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw_views.csv").resolve()

dataframe_views.to_csv(os.path.join(full_path))

In [ ]:
### Assign category (bad, good, great) for score

score_bucketing_all = lambda x: 'bad' if x < 0 else 'good' if (x >= 1 and x <= 6) else 'great' if x >= 7 else None

dataframe_views['score_cat_all'] = dataframe_views['answer_score'].apply(score_bucketing_all)

In [ ]:
print("{} unique values in column".format('score_cat_all'))
print("{}".format(dataframe_views['score_cat_all'].unique()),"\n")

In [ ]:
### Assign category (good, great) for score; ignore negative and zero scores

score_bucketing_positive = lambda x: 'good' if (x >= 1 and x <= 6) else 'great' if x >= 7 else None

dataframe_views['score_cat_positive'] = dataframe_views['answer_score'].apply(score_bucketing_positive)

In [ ]:
print("{} unique values in column".format('score_cat_positive'))
print("{}".format(dataframe_views['score_cat_positive'].unique()),"\n")

In [ ]:
### Look at the data to understand the type and missing values

print(dataframe_views.info())

In [ ]:
display(dataframe_views)

In [ ]:
### Sample equal amounts of data rows for "bad", "good" and "great" answers from main dataset

dataframe_sampled_all = dataframe_views.groupby('score_cat_all', group_keys=False).apply(lambda x: x.sample(n=13000, random_state = 1)).reset_index(drop=True)

In [ ]:
### Check if equal sampling of "bad", "good", and "great" answers was successful

dataframe_sampled_all.score_cat_all.value_counts()

In [ ]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw_views_sampled_all.csv").resolve()

dataframe_sampled_all.to_csv(os.path.join(full_path))

In [ ]:
### Sample only one answer randomly per stackoverflow question

df_all_final = dataframe_sampled_all.groupby('question_id', group_keys=False).apply(lambda x: x.sample(n = 1, random_state = 1)).reset_index(drop=True)

In [ ]:
### Sample equal amounts of data rows for "bad", "good" and "great" answers from main dataset

df_all_final = df_all_final.groupby('score_cat_all', group_keys=False).apply(lambda x: x.sample(n=10000, random_state = 1)).reset_index(drop=True)

In [ ]:
### Check if classes "bad", "good", and "great" are still roughly balanced

df_all_final.score_cat_all.value_counts()

In [ ]:
### Save sample of dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw_finalsample_all.csv").resolve()
df_all_final.to_csv(os.path.join(full_path))

In [ ]:
### Sample equal amounts of data rows for "good" and "great" answers from main dataset

dataframe_sampled_positive = dataframe_views.groupby('score_cat_positive', group_keys=False).apply(lambda x: x.sample(n=18000, random_state = 1)).reset_index(drop=True)

In [ ]:
### Check if equal sampling of classes "good" and "great" was successful

dataframe_sampled_positive.score_cat_positive.value_counts()

In [ ]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw_views_sampled_positive.csv").resolve()

dataframe_sampled_positive.to_csv(os.path.join(full_path))

In [ ]:
### Sample only one answer randomly per stackoverflow question

df_positive_final = dataframe_sampled_positive.groupby('question_id', group_keys=False).apply(lambda x: x.sample(n = 1, random_state = 1)).reset_index(drop=True)

In [ ]:
### Sample equal amounts of data rows for "good" and "great" answers from main dataset

df_positive_final = df_positive_final.groupby('score_cat_positive', group_keys=False).apply(lambda x: x.sample(n=15000, random_state = 1)).reset_index(drop=True)

In [ ]:
### Check if classses "good" and "great" are still roughly balanced

df_positive_final.score_cat_positive.value_counts()

In [ ]:
### Save dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../data/raw/stackoverflow_raw_finalsample_positive.csv").resolve()

df_positive_final.to_csv(os.path.join(full_path))